In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [11]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

In [12]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = F.max_pool2d(F.relu(self.conv1(t)), kernel_size=2, stride=2)

        t = F.max_pool2d(F.relu(self.conv2(t)), kernel_size=2, stride=2)

        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

In [13]:
def num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [19]:
network = Network()
batch = next(iter(train_loader))
optimiser = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(3):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: #looping each image in the batch
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        total_loss += loss.item()
        total_correct += num_correct(preds, labels)

    print('epoch:', epoch , 'loss:', total_loss, 'total correct:', total_correct)

epoch: 0 loss: 345.8421598523855 total correct: 46675
epoch: 1 loss: 227.41074959933758 total correct: 51542
epoch: 2 loss: 208.94825959205627 total correct: 52212


In [20]:
52212 / len(train_set) #accuracy percentage

0.8702